In [1]:
import pandas as pd
from ipynb.fs.full.quick_functions import *

countries = ['AUS', 'BRA','CAN', 'CHE', 'CHL','CHN', 'COL','CZE', 'GBR', 'HUN', 'IDN','IND',
             'KOR','MEX', 'NOR', 'NZL','POL', 'USA', 'ZAF']

file_path = 'Downloads/OECD_CPI_KEY.xlsx'

# Read all sheets into a dictionary
xls = pd.ExcelFile(file_path)
sheet_list = []

for sheet_name in xls.sheet_names:
    sheet_data = pd.read_excel(file_path, sheet_name=sheet_name)
    sheet_list.append(sheet_data)
    
country = sheet_list[2]
Subject_CPI = sheet_list[3]
Subject_HICP = sheet_list[4]
Measure = sheet_list[5]

#save into dictionaries all the keys
country_dict = dict(zip(country["Country Code"].apply(lambda x: x.strip()),country["Country Name"]))
Subject_CPI = dict(zip(Subject_CPI.iloc[:,0],Subject_CPI.iloc[:,1]))
Subject_HICP = dict(zip(Subject_HICP.iloc[:,0],Subject_HICP.iloc[:,1]))
Measure = dict(zip(Measure["Measure Code"],Measure["Measure name"]))
all_acronyms = {**country_dict, **Subject_CPI, **Subject_HICP, **Measure}

df = pd.read_csv("Full_PRICES_CPI.csv")
df.head()

,LOCATION,SUBJECT,MEASURE,FREQUENCY,TIME_PERIOD,value
0,AUS,CP010000,GP,A,2000,0.272991
1,AUS,CP010000,GP,A,2001,6.280433
2,AUS,CP010000,GP,Q,2000-Q1,-0.063741
3,AUS,CP010000,GP,Q,2000-Q2,0.965745
4,AUS,CP010000,GP,Q,2000-Q3,-1.186929


In [5]:
#select CPI items for a specific country
country = "MEX"
major_cpi_items = ["CPALTT01","CPGREN01",'CPGRLE01','CPGRSE01','CPGRGO01','CPGRHO01','CPGRLH02']
filtered_df = df.query(f"SUBJECT == {major_cpi_items} and FREQUENCY == 'M' and MEASURE == 'CTGY' and LOCATION == 'MEX'")#[["SUBJECT","TIME_PERIOD","value"]] 
#filtered_df[filtered_df.duplicated()]
pivoted_df = filtered_df.pivot_table(index="TIME_PERIOD",columns="SUBJECT",values="value",aggfunc='mean')
final_df = pivoted_df.rename(columns=Subject_CPI)
fig = px.line(final_df)
fig.update_layout(xaxis_title=None,yaxis_title="Contributions to Headline",title=country_dict.get(country))
fig.show()

<IPython.core.display.Javascript object>

In [10]:
#Same CPI item for all countries
cpi_item = "CPGREN01"
country = "MEX"
measure = "GY"

filtered_df2 = df.query(f"SUBJECT == '{cpi_item}' and FREQUENCY == 'M' and MEASURE == '{measure}'")#[["SUBJECT","TIME_PERIOD","value"]] 
pivoted_df2 = filtered_df2.pivot_table(index="TIME_PERIOD",columns="LOCATION",values="value",aggfunc="mean") \
    .iloc[-130:]
pivoted_df2 = delete_outliers(pivoted_df2) #delete outliers
select_countries = [x for x in pivoted_df2.columns if x in countries]
select_countries
#for grey vs red highlights
myindex = select_countries.index(country)
my_list = ['lightgray'] * (len(select_countries))
my_list[myindex] = 'red'
fig = px.line(pivoted_df2[select_countries].rename(columns=all_acronyms),color_discrete_sequence=my_list)
fig.update_layout(xaxis_title=None,yaxis_title=all_acronyms[cpi_item],plot_bgcolor='LightCyan')
fig.show()

<IPython.core.display.Javascript object>

In [4]:
all_acronyms

{'AUS': 'Australia',
 'AUT': 'Austria',
 'BEL': 'Belgium',
 'CAN': 'Canada',
 'CHL': 'Chile',
 'CZE': 'Czech Republic',
 'DNK': 'Denmark',
 'EST': 'Estonia',
 'FIN': 'Finland',
 'FRA': 'France',
 'DEU': 'Germany',
 'GRC': 'Greece',
 'HUN': 'Hungary',
 'ISL': 'Iceland',
 'IRL': 'Ireland',
 'ISR': 'Israel',
 'ITA': 'Italy',
 'JPN': 'Japan',
 'KOR': 'Korea',
 'LVA': 'Latvia',
 'LUX': 'Luxembourg',
 'MEX': 'Mexico',
 'NLD': 'Netherlands',
 'NZL': 'New Zealand',
 'NOR': 'Norway',
 'POL': 'Poland',
 'PRT': 'Portugal',
 'SVK': 'Slovak Republic',
 'SVN': 'Slovenia',
 'ESP': 'Spain',
 'SWE': 'Sweden',
 'CHE': 'Switzerland',
 'TUR': 'Turkey',
 'GBR': 'the United Kingdom',
 'USA': 'United States',
 'EA19': 'Euro area (19 countries)',
 'EU28': 'European Union (28 countries)',
 'G-7': 'G7',
 'OECDE': 'OECD-Europe',
 'G-20': 'G20',
 'OECD': 'OECD-Total',
 'ARG': 'Argentina',
 'BRA': 'Brazil',
 'CHN': "China (People's Republic of)",
 'COL': 'Colombia',
 'CRI': 'Costa Rica',
 'IND': 'India',
 'IDN': '